# Init

In [1]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import ner

In [2]:
import numpy as np

def cosine_similarity(vec1, vec2):
    # Compute the dot product of the vectors
    dot_product = np.dot(vec1, vec2)
    # Compute the magnitude (norm) of each vector
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    # Return the cosine similarity
    return dot_product / (norm_vec1 * norm_vec2)

# Example
vector1 = np.array([1, 2, 3])
vector2 = np.array([4, 5, 6])
similarity = cosine_similarity(vector1, vector2)
print(f"Cosine Similarity: {similarity}")

Cosine Similarity: 0.9746318461970762


# Test: Table Description & Query

In [3]:
tables = {
    "ConstantDB": {
        "intro": "作为常量库，包含各种基础性、标准化的参照表，如证券主表（记录股票、基金、债券等基础信息）、系统常量表（各种常量分类）、国家城市代码表、交易日表等。"
    },
    "AStockBasicInfoDB": {
        "intro": "主要收录 A 股上市公司的基础信息，如公司概况、名称变更、经营范围与行业变更等，帮助用户快速了解公司基本背景。"
    },
    "AStockIndustryDB": {
        "intro": "聚焦 A 股上市公司的行业划分、概念板块、行业衍生指标以及行业估值、财务指标等，能反映公司所处行业、行业变动以及行业层面的综合表现。"
    },
    "AStockOperationsDB": {
        "intro": "记录 A 股上市公司在供应链端的供销数据、人力资源数据等，反映企业在业务运营层面的情况，包括供应商、客户信息、职工构成等。"
    },
    "AStockShareholderDB": {
        "intro": "涵盖 A 股上市公司股东与股本结构、实际控制人、股权变动、股东大会出席情况等治理层面的信息，用于跟踪公司所有权结构及其历史变动。"
    },
    "AStockFinanceDB": {
        "intro": "着重于 A 股上市公司的财务报表与融资分红数据，包括资产负债表、利润表、现金流量表、增发配股分红、主营业务构成等财务核心信息。"
    },
    "AStockMarketQuotesDB": {
        "intro": "存储 A 股市场行情相关数据，包括日行情、成交量指标、资金流向指标、技术形态等，为分析股价走势、交易活跃度和资金进出提供依据。"
    },
    "AStockEventsDB": {
        "intro": "收录 A 股上市公司在公告、临时公告中披露的重大事件，如担保、借贷、诉讼仲裁、委托理财、资产重组、重大合同等。"
    },
    "HKStockDB": {
        "intro": "专门用于港股（香港上市公司）相关的基础信息与数据，包括公司概况、员工变动、港股行情表现等。"
    },
    "USStockDB": {
        "intro": "面向美国市场上市公司的基础信息和交易行情，包含公司概况、股票每日收盘行情等。"
    },
    "PublicFundDB": {
        "intro": "公募基金相关数据库，记录基金概况、产品名称、管理人概况、基金分红等，适用于各种公募基金研究和投资分析。"
    },
    "CreditDB": {
        "intro": "专注于违规惩处信息，记录相关主体（个人、机构）在证券市场的处罚信息，包括处罚机构、违规类型、涉及金额等。"
    },
    "IndexDB": {
        "intro": "收录市场上主要指数的基础信息和成份数据，如指数发布机构、计算方法、成份股列表及其变动历史。"
    },
    "InstitutionDB": {
        "intro": "包含市场主要机构（证券公司、信托公司、保险公司等）的基本信息，以及聚源舆情与事件体系相关的配套表，帮助用户研究机构背景或事件舆情。"
    }
}

In [4]:
from zhipuai import ZhipuAI

api_key = chat.load_api_key()

def compute_embedding(text: str) -> list:
    client = ZhipuAI(api_key=api_key) 
    response = client.embeddings.create(
        model="embedding-3", #填写需要调用的模型编码
        input=[
            text,
        ],
    )
    res = json.loads(response.to_json())['data'][0]['embedding']

    return res

In [5]:
for k, v in tables.items():
    text = v['intro']
    tables[k]['embedding'] = compute_embedding(text)

In [6]:
query = "化工纳入过多少个子类概念？"
e = compute_embedding(query)

In [7]:
# {query: {embedding: , top_10: ,}}
cases = {}

In [8]:
def run_test(query: str):

    e = compute_embedding(query)

    for k, v in tables.items():
        tables[k]['similarity'] = cosine_similarity(tables[k]['embedding'], e)

    # 1. Sort by 'similarity' in descending order
    sorted_data = sorted(
        tables.items(), 
        key=lambda item: item[1]['similarity'], 
        reverse=True
    )

    # 2. Slice the top 10
    top_10 = sorted_data[:10]
    top_10_res = []

    # 3. Display the table name and its intro
    for table_name, info in top_10:
        top_10_res.append(f"{table_name}: {info['intro']} (similarity={info['similarity']})")

    cases.update({query: {"embedding": e, "top_10": top_10_res}})

    print('\n'.join(top_10_res))

def show_case(data: dict, top: int):

    for k, v in data.items():
        tmp = '\n'.join(v['top_10'][:top])
        print(f"{k}:\n{tmp}\n")

In [9]:
run_test('深科技在2021年12月24日的交易数据如何?具体包括收盘价、成交量、换手率，保留2位小数。')

USStockDB: 面向美国市场上市公司的基础信息和交易行情，包含公司概况、股票每日收盘行情等。 (similarity=0.3784781831258849)
AStockOperationsDB: 记录 A 股上市公司在供应链端的供销数据、人力资源数据等，反映企业在业务运营层面的情况，包括供应商、客户信息、职工构成等。 (similarity=0.3463383196265334)
AStockFinanceDB: 着重于 A 股上市公司的财务报表与融资分红数据，包括资产负债表、利润表、现金流量表、增发配股分红、主营业务构成等财务核心信息。 (similarity=0.3442859793662114)
AStockBasicInfoDB: 主要收录 A 股上市公司的基础信息，如公司概况、名称变更、经营范围与行业变更等，帮助用户快速了解公司基本背景。 (similarity=0.3344667722486298)
AStockShareholderDB: 涵盖 A 股上市公司股东与股本结构、实际控制人、股权变动、股东大会出席情况等治理层面的信息，用于跟踪公司所有权结构及其历史变动。 (similarity=0.33085064862278246)
AStockMarketQuotesDB: 存储 A 股市场行情相关数据，包括日行情、成交量指标、资金流向指标、技术形态等，为分析股价走势、交易活跃度和资金进出提供依据。 (similarity=0.3273826957838641)
HKStockDB: 专门用于港股（香港上市公司）相关的基础信息与数据，包括公司概况、员工变动、港股行情表现等。 (similarity=0.3159575666689483)
IndexDB: 收录市场上主要指数的基础信息和成份数据，如指数发布机构、计算方法、成份股列表及其变动历史。 (similarity=0.30971171338450704)
AStockIndustryDB: 聚焦 A 股上市公司的行业划分、概念板块、行业衍生指标以及行业估值、财务指标等，能反映公司所处行业、行业变动以及行业层面的综合表现。 (similarity=0.28534446126465113)
AStockEventsDB: 收录 A 股上市公司在公告、临时公告中披露的重大事件，如担保、借贷、诉讼

In [10]:
show_case(cases, 5)

深科技在2021年12月24日的交易数据如何?具体包括收盘价、成交量、换手率，保留2位小数。:
USStockDB: 面向美国市场上市公司的基础信息和交易行情，包含公司概况、股票每日收盘行情等。 (similarity=0.3784781831258849)
AStockOperationsDB: 记录 A 股上市公司在供应链端的供销数据、人力资源数据等，反映企业在业务运营层面的情况，包括供应商、客户信息、职工构成等。 (similarity=0.3463383196265334)
AStockFinanceDB: 着重于 A 股上市公司的财务报表与融资分红数据，包括资产负债表、利润表、现金流量表、增发配股分红、主营业务构成等财务核心信息。 (similarity=0.3442859793662114)
AStockBasicInfoDB: 主要收录 A 股上市公司的基础信息，如公司概况、名称变更、经营范围与行业变更等，帮助用户快速了解公司基本背景。 (similarity=0.3344667722486298)
AStockShareholderDB: 涵盖 A 股上市公司股东与股本结构、实际控制人、股权变动、股东大会出席情况等治理层面的信息，用于跟踪公司所有权结构及其历史变动。 (similarity=0.33085064862278246)



# TEST: Query & Market

In [3]:
from zhipuai import ZhipuAI

api_key = chat.load_api_key()

def compute_embedding(text: str) -> list:
    client = ZhipuAI(api_key=api_key) 
    response = client.embeddings.create(
        model="embedding-3", #填写需要调用的模型编码
        input=[
            text,
        ],
    )
    res = json.loads(response.to_json())['data'][0]['embedding']

    return res

In [6]:
us = compute_embedding('US，美股，美国')
hk = compute_embedding('HK，港股，香港')
cn = compute_embedding('CN，A股，中国')

query = compute_embedding('波司登在港股的股票代码是什么？')

cosine_similarity(us, query)

cosine_similarity(hk, query)

cosine_similarity(cn, query)

0.526263222125305

0.6100846185343104

0.5660880718358094

In [8]:
import numpy as np

# 使用 savez 存储多个数组，并为每个数组指定名字
np.savez('embeddings.npz', CN=cn, US=us, HK=hk)

In [3]:
ner.match_market_through_embedding('波司登在港股的股票代码是什么？')

'HK'